In [ ]:
#pip install torch_geometric > /dev/null
#pip install rdflib

#!pip install torch  > /dev/null
#!pip install scikit-learn > /dev/null


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import  time


import torch
import random
import csv


In [3]:
from src.gnn import *
from src.gnn_utils import *

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1. Data

In [9]:
filename = 'cora_deduped_v2'
to_evaluate = [rdflib.term.URIRef('http://www.w3.org/2002/07/owl#sameAs')]

In [10]:
(data, train_pos_edge_adj, test_pos_edge_adj, _, nodes_dict, relations_dict) = read_data(filename, to_evaluate, round=1, same_as_fraction_used_training=1)
same_as_index = relations_dict[rdflib.term.URIRef('http://www.w3.org/2002/07/owl#sameAs')]
relation_ratio = data['train_edge_type'].bincount() / data['train_edge_type'].size(0)
num_nodes = len(nodes_dict)
num_relations = len(relations_dict.keys())
train_loader, test_loader, validation_loader = get_dataloaders(data, to_evaluate, batch_size_train=int(data.train_pos_edge_index.size(1)/4), batch_size_test='all')

Triplets found: 33351


In [11]:
data

Data(edge_index=[2, 33351], edge_type=[33351], test_pos_edge_index=[2, 339], test_edge_type=[339], train_pos_edge_index=[2, 33012], train_edge_type=[33012], test_edge_eval_masks=[1, 339], train_edge_eval_masks=[1, 33012])

# 2. GNN

**Train**

In [12]:
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)
torch.use_deterministic_algorithms(False)

In [7]:
st = time.time()
model = GNN(num_nodes=num_nodes, num_relations=num_relations, num_layers=5, hidden_channels=500, learning_rate=1e-2, dropout_prob=0.2, regularization_val=1e-2, same_as_index=same_as_index, relation_ratio=relation_ratio)
epochs = 10

for epoch in range(epochs):
    loss = model.train(train_loader, data['train_pos_edge_index'])
    
    print(f'Epoch: {epoch}, Loss: {loss:.4f}')
    
    hits1_train, hits10_train, mr_train, mrr_train = model.eval(train_loader, train_pos_edge_adj, to_evaluate)
    for i, relation in enumerate(to_evaluate):
        print(f"\t{relation.rsplit('#', 1)[1]} --> Train_Hits@1: {hits1_train[i]:.3f}, Train_Hits@10: {hits10_train[i]:.3f}, Train_MR: {mr_train[i]:.3f}, Train_MRR: {mrr_train[i]:.3f}")
        _, _, _, _, _, _, precision_train, recall_train, f1_train = model.eval_PRF(train_loader, i)
        print(f"\t{relation.rsplit('#', 1)[1]} --> Train_Precision: {precision_train:.3f}, Train_Recall: {recall_train:.3f}, Train_F1: {f1_train:.3f}")
        

    print(f'Epoch: {epoch}, Loss: {loss:.4f}')
    hits1_test, hits10_test, mr_test, mrr_test = model.eval(test_loader, test_pos_edge_adj, to_evaluate)
    for i, relation in enumerate(to_evaluate):
        print(f"\t{relation.rsplit('#', 1)[1]} --> Test_Hits@1: {hits1_test[i]:.3f}, Test_Hits@10: {hits10_test[i]:.3f} Test_MR: {mr_test[i]:.3f}, Test_MRR: {mrr_test[i]:.3f}")
        _, _, _, _, _, _, precision_test, recall_test, f1_test = model.eval_PRF(test_loader, i)
        print(f"\t{relation.rsplit('#', 1)[1]} --> Test_Precision: {precision_test:.3f}, Test_Recall: {recall_test:.3f}, Test_F1: {f1_test:.3f}")
        

et = time.time()
elapsed_time = et - st
print(f'Run time: {elapsed_time:.0f} seconds, {elapsed_time/60:.0f} minutes')

training:   0%|          | 0/5 [00:01<?, ?it/s]


KeyboardInterrupt: 

In [13]:
model.device

device(type='cuda')

## Same loop, but writing the results to a file 

In [15]:
st = time.time()
model = GNN(num_nodes=num_nodes, num_relations=num_relations, num_layers=5, hidden_channels=400, learning_rate=1e-2, dropout_prob=0.2, regularization_val=1e-2, same_as_index=same_as_index, relation_ratio=relation_ratio)
epochs = 100

for relation in to_evaluate:
    filename_train = f'../results/{filename}_train_v2.csv'
    filename_test = f'../results/{filename}_test_v2.csv'

with open(filename_train, 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    headerList = ['epoch', 'loss']
    for relation in to_evaluate:
        headerList.extend([f'{relation.rsplit("#", 1)[1]}_train_hits@1',f'{relation.rsplit("#", 1)[1]}_train_hits@10',f'{relation.rsplit("#", 1)[1]}_train_MR',f'{relation.rsplit("#", 1)[1]}_train_MRR', f'{relation.rsplit("#", 1)[1]}_train_precision',f'{relation.rsplit("#", 1)[1]}_train_recall',f'{relation.rsplit("#", 1)[1]}_train_f1'])
    csvwriter.writerow(headerList)
    
with open(filename_test, 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    headerList = ['epoch']
    for relation in to_evaluate:
        headerList.extend([f'{relation.rsplit("#", 1)[1]}_test_hits@1',f'{relation.rsplit("#", 1)[1]}_test_hits@10',f'{relation.rsplit("#", 1)[1]}_test_MR',f'{relation.rsplit("#", 1)[1]}_test_MRR', f'{relation.rsplit("#", 1)[1]}_test_precision',f'{relation.rsplit("#", 1)[1]}_test_recall',f'{relation.rsplit("#", 1)[1]}_test_f1'])
    csvwriter.writerow(headerList)

for epoch in range(epochs):
    print(f'Epoch: {epoch}/{epochs}')
    loss = model.train(train_loader, data['train_pos_edge_index'])
    
    hits1_train, hits10_train, mr_train, mrr_train = model.eval(train_loader, train_pos_edge_adj, to_evaluate)
    row = [epoch, loss]
    for i, relation in enumerate(to_evaluate):
        _, _, _, _, _, _, precision_train, recall_train, f1_train = model.eval_PRF(train_loader, i)
        row.extend([hits1_train[i].item(), hits10_train[i].item(), mr_train[i].item(), mrr_train[i].item(), precision_train, recall_train, f1_train])
        
    
    with open(filename_train, 'a') as csvfile:     
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(row)

    hits1_test, hits10_test, mr_test, mrr_test = model.eval(test_loader, test_pos_edge_adj, to_evaluate)
    row = [epoch]
    for i, relation in enumerate(to_evaluate):
        _, _, _, _, _, _, precision_test, recall_test, f1_test = model.eval_PRF(test_loader, i)
        row.extend([hits1_test[i].item(), hits10_test[i].item(), mr_test[i].item(), mrr_test[i].item(), precision_test, recall_test, f1_test])
    
    with open(filename_test, 'a') as csvfile:     
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(row)

et = time.time()
elapsed_time = et - st
print(f'Run time: {elapsed_time:.0f} seconds, {elapsed_time/60:.0f} minutes')

Epoch: 0/100


evaluating: 100%|██████████| 1/1 [00:00<00:00,  4.66it/s]


Epoch: 1/100


evaluating: 100%|██████████| 1/1 [00:00<00:00,  4.27it/s]


Epoch: 2/100


...

Epoch: 98/100


evaluating: 100%|██████████| 1/1 [00:00<00:00,  4.69it/s]


Epoch: 99/100


evaluating: 100%|██████████| 1/1 [00:00<00:00,  4.70it/s]

Run time: 744 seconds, 12 minutes


In [7]:
st = time.time()
model = GNN(num_nodes=num_nodes, num_relations=num_relations, num_layers=3, hidden_channels=200, learning_rate=5e-3, dropout_prob=0.2, regularization_val=1e-2, same_as_index=same_as_index, relation_ratio=relation_ratio, node_from_ontology=node_from_ontology, nodes_ontology_1=nodes_ontology_1, nodes_ontology_2=nodes_ontology_2)
epochs = 100

for relation in to_evaluate:
    filename_train = f'../results/{filename}_train_results_round3.csv'
    filename_test = f'../results/{filename}_test_results_round3.csv'

with open(filename_train, 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    headerList = ['epoch', 'loss']
    for relation in to_evaluate:
        headerList.extend([f'{relation.rsplit("#", 1)[1]}_train_hits@1',f'{relation.rsplit("#", 1)[1]}_train_hits@10',f'{relation.rsplit("#", 1)[1]}_train_MR',f'{relation.rsplit("#", 1)[1]}_train_MRR', f'{relation.rsplit("#", 1)[1]}_train_precision',f'{relation.rsplit("#", 1)[1]}_train_recall',f'{relation.rsplit("#", 1)[1]}_train_f1'])
    csvwriter.writerow(headerList)
    
with open(filename_test, 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    headerList = ['epoch']
    for relation in to_evaluate:
        headerList.extend([f'{relation.rsplit("#", 1)[1]}_test_hits@1',f'{relation.rsplit("#", 1)[1]}_test_hits@10',f'{relation.rsplit("#", 1)[1]}_test_MR',f'{relation.rsplit("#", 1)[1]}_test_MRR', f'{relation.rsplit("#", 1)[1]}_test_precision',f'{relation.rsplit("#", 1)[1]}_test_recall',f'{relation.rsplit("#", 1)[1]}_test_f1'])
    csvwriter.writerow(headerList)

for epoch in range(epochs):
    print(f'Epoch: {epoch}/{epochs}')
    loss = model.train(train_loader, data['train_pos_edge_index'])
    
    hits1_train, hits10_train, mr_train, mrr_train = model.eval(train_loader, train_pos_edge_adj, to_evaluate)
    row = [epoch, loss]
    for i, relation in enumerate(to_evaluate):
        _, _, _, _, _, _, precision_train, recall_train, f1_train = model.eval_PRF(train_loader, i)
        row.extend([hits1_train[i].item(), hits10_train[i].item(), mr_train[i].item(), mrr_train[i].item(), precision_train, recall_train, f1_train])
        
    
    with open(filename_train, 'a') as csvfile:     
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(row)

    hits1_test, hits10_test, mr_test, mrr_test = model.eval(test_loader, test_pos_edge_adj, to_evaluate)
    row = [epoch]
    for i, relation in enumerate(to_evaluate):
        _, _, _, _, _, _, precision_test, recall_test, f1_test = model.eval_PRF(test_loader, i)
        row.extend([hits1_test[i].item(), hits10_test[i].item(), mr_test[i].item(), mrr_test[i].item(), precision_test, recall_test, f1_test])
    
    with open(filename_test, 'a') as csvfile:     
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(row)

et = time.time()
elapsed_time = et - st
print(f'Run time: {elapsed_time:.0f} seconds, {elapsed_time/60:.0f} minutes')

Epoch: 0/100


evaluating: 100%|██████████| 5/5 [00:16<00:00,  3.24s/it]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
evaluating: 100%|██████████| 1/1 [00:03<00:00,  3.72s/it]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch: 1/100


evaluating: 100%|██████████| 1/1 [00:03<00:00,  3.76s/it]


Epoch: 2/100


...

Epoch: 98/100


evaluating: 100%|██████████| 1/1 [00:03<00:00,  3.10s/it]


Epoch: 99/100


evaluating: 100%|██████████| 1/1 [00:03<00:00,  3.81s/it]

Run time: 2265 seconds, 38 minutes


In [ ]:
#try with originsl param
#remove duplicated from data 